In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import re
import functools
import string
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import time
pd.set_option('display.max_rows', 300)

In [3]:
# import os
dir_path = '/content/drive/MyDrive/CSS_project/data/'
# os.chdir(dir_path)
file_name = '/content/drive/MyDrive/CSS_project/data/queens_death_place2.json'

# Making sense of the data

To make sense of the data, see documentation
https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview

`Tweet` object:
- Retweets can be distinguished from typical Tweets by the existence of a `retweeted_status` attribute
- `quoted_status_id`: This field only surfaces when the Tweet is a quote Tweet. This field contains the integer value Tweet ID of the quoted Tweet
- `quote_count`, `reply_count`, `retweet_count`, `favorite_count` (like count)
- `entities` object contains parsed information about the tweet (`hashtags`, `urls`, `user_mentions`, `symbols`; `media`, `polls`)
- Top-level entites do not reflect Quote details, so we should rely on the nested `extended_tweet` object for Quote Tweets.
- `geo`: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/geo

`Author` object also has `location`

What is `context_annotations`? https://developer.twitter.com/en/docs/twitter-api/annotations/overview

- `referenced_tweets`: `replied_to`, `retweets`

***Questions***
1. How do we handle retweets and quoted tweets?
2. Maybe we can do something with the hashtags?
3. There are many urls. We should make use of them (e.g. title, description)
4. Infer occupation from description?
5. Which location to use?
  - Many tweets don't have location. Conjecture: most account with location and public figures/institutions.
  - Author locations need disambiguation


Separate retweets for now

1. Discard non-English tweets and remove non-English characters, also get rid of the __twarc field

In [ ]:
# with open(file_name, 'r') as f:
#   f_write = open(dir_path+'all_en_tweets.json', 'w')
#   num_tweets = 0
#   while True:
#     try: # goes through every line
#       t = json.loads(f.readline()) # a tweet
#       if t['lang']!='en': # discard non-English tweets
#           continues
#       del t['__twarc']  # delete query information
#       t['text'] = re.sub(f'[^A-Za-z0-9\s{string.punctuation}]', '', t['text']) # remove non-English chars
#       f_write.write(json.dumps(t)+'\n')
#       num_tweets += 1
#     except:
#       break
#   f_write.close()
# print(num_tweets)

4432930


In [ ]:
# referenced_types = set()
# with open(dir_path+'all_en_tweets.json', 'r') as f:
#   while True:
#     line = f.readline()
#     try:
#       t = json.loads(line)
#       if 'referenced_tweets' in t:
#         assert len(t['referenced_tweets'])==1
#         referenced_types |= set([t['referenced_tweets'][0]['type']])
#     except:
#       break
# referenced_types      

{'quoted', 'replied_to', 'retweeted'}

2. Split all_en_tweets.json into tweets.json and retweets.json
Note that this classification is not very rigorous. It is better to look at `referenced_tweets` = `replied_to`, `retweeted`, `quoted`

In [ ]:
# # Took 30 mins
# f_tw = open(dir_path + 'tweets.json', 'w')  # not retweets
# f_rt = open(dir_path + 'retweets.json', 'w') # retweets
# with open(dir_path + 'all_en_tweets.json', 'r') as f:
#   while True:
#     try: # goes through every line
#       t = json.loads(f.readline()) # a tweet
#       if bool(re.match('^RT', t['text'])) and 'referenced_tweets' in t: # text of retweets start with RT
#         f_rt.write(json.dumps(t)+'\n')
#       else:
#         f_tw.write(json.dumps(t)+'\n')
#     except:
#       break
# f_rt.close()
# f_tw.close()

In [ ]:
# # how many lines (i.e. tweets) are there in the file
# !wc -l 'queens_death.json' # 
# !wc -l 'all_en_tweets.json' # 
# !wc -l 'tweets.json'    # 
# !wc -l 'retweets.json'   # 

^C


## Create an `authors.json` file for later use 
(note that it doesn't contain authors of retweeted tweets)

In [ ]:
# # Took 16 mins
# authors_added = set()
# with open(dir_path + 'all_en_tweets.json', 'r') as f:
#   f_a = open(dir_path+'authors.json', 'w')
#   while True:
#     try: # goes through every line
#       t = json.loads(f.readline()) # a tweet
#       if t['author']['id'] in authors_added: # if author is already added
#         continue
#       # ============== Add info for M3Inference =================
#       # creating 'screen_name'
#       t['author']['screen_name'] = t['author']['username']
#       # creating 'profile_image_url_https'
#       t['author']['profile_image_url_https'] = t['author']['profile_image_url']
#       # adding lang
#       t['author']['lang'] = 'en'
#       # adding id_str
#       t['author']['id_str'] = str(t['author']['id'])
#       # adding default_profile_image
#       if t['author']["profile_image_url"] == "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png":
#          t['author']["default_profile_image"] = True
#       else:
#          t['author']["default_profile_image"] = False
#       # ============================================================
#       a = t['author']
#       authors_added.add(a['id'])
#       f_a.write(json.dumps(a)+'\n')
#     except:
#       break
#   f_a.close()

In [ ]:
def read_tweets_from(filename, num_rows=None):
  tweets = []
  with open(dir_path+filename, 'r') as f:
    if num_rows is None: # read the entire file
      while True:
        try:
          tweets.append(json.loads(f.readline()))
        except:
          break
    else:
      for i in range(num_rows):
        tweets.append(json.loads(f.readline()))
  return tweets

## Exploration (Don't use this. See code in the next Exploration section)

In [ ]:
# read all tweets from tweets.json
tweets = read_tweets_from('tweets.json')

In [ ]:
# # look for info about geolocation
print(functools.reduce(set.union, [set(t.keys()) for t in tweets])) # geo 
# print(functools.reduce(set.union, [set(t['author'].keys()) for t in tweets])) # location

{'lang', 'author', 'public_metrics', 'attachments', 'reply_settings', 'referenced_tweets', 'conversation_id', 'created_at', 'text', 'source', 'possibly_sensitive', 'in_reply_to_user', 'context_annotations', 'id', 'entities', 'withheld', 'author_id', 'geo', 'in_reply_to_user_id'}


In [ ]:
# # only a small portion of tweets have location
# len([x for x in tweets if 'geo' in x]), len(tweets)

In [ ]:
# # most authors do have locations
# len([x for x in tweets if 'location' in x['author']]), len(tweets)

In [ ]:
# # tweet locations
# print(functools.reduce(set.union, [set(t['geo'].keys()) for t in tweets if 'geo' in t]))
# countries = [t['geo']['country'] for t in tweets if 'geo' in t]
# countries = pd.Series(countries).value_counts()

In [ ]:
# import plotly.express as px
# fig = px.pie(values=countries, names=countries.index)
# fig.update_layout(title="Tweet location by country (excluding retweets)", title_x=0.5)
# fig.update_traces(textposition='inside',textinfo='percent+label', showlegend=False)

For layered pie chart see https://towardsdev.com/pie-charts-in-python-matplotlib-seaborn-plotly-f60995160c33

In [ ]:
# tweets_locations = {t['geo'] for t in tweets if 'geo' in t}
# author_locations = {t['author']['location'] for t in tweets if 'location' in t['author']}
# # We might need to use google maps to figure out author locations ...

In [ ]:
# {t['reply_settings'] for t in tweets}

# Build the database

## Create Tables

Tables
1. Authors
2. Tweets
3. Retweets

In [104]:
import sqlite3
con = sqlite3.connect(dir_path+"queen.db", timeout=10)

In [ ]:
# con.execute("""DROP TABLE IF EXISTS Authors""")
# con.execute("""CREATE TABLE IF NOT EXISTS Authors (
#             author_id TEXT NOT NULL PRIMARY KEY UNIQUE,
#             name TEXT,
#             username TEXT,
#             description TEXT,
#             location TEXT,
#             verified INTEGER,
#             followers_count INTEGER,
#             following_count INTEGER,
#             tweet_count INTEGER,
#             listed_count INTEGER,
#             profile_image_url TEXT,
#             url TEXT);
# """)

In [ ]:
# con.execute("""DROP TABLE IF EXISTS Tweets""")
# con.execute("""CREATE TABLE IF NOT EXISTS Tweets (
#             tweet_id TEXT NOT NULL PRIMARY KEY UNIQUE,
#             author_id TEXT NOT NULL,
#             tweet_text TEXT NOT NULL,
#             country TEXT,
#             retweet_count INTEGER,
#             reply_count INTEGER,
#             like_count INTEGER,
#             quote_count INTEGER,    
#             FOREIGN KEY (author_id) REFERENCES Authors(author_id)
#         );
# """)
# # Note that we dropped `refereence_tweets` for now. We can come back later ...

In [ ]:
# con.execute("""DROP TABLE IF EXISTS Retweets""")
# con.execute("""CREATE TABLE Retweets (
#             tweet_id TEXT NOT NULL PRIMARY KEY UNIQUE,
#             original_tweet_id TEXT NOT NULL,
#             author_id TEXT NOT NULL,
#             original_author_id TEXT NOT NULL,
#             country TEXT,
#             retweet_count INTEGER,
#             reply_count INTEGER,
#             like_count INTEGER,
#             quote_count INTEGER,    
#             FOREIGN KEY (author_id) REFERENCES Authors(author_id),
#             FOREIGN KEY (original_author_id) REFERENCES Authors(author_id),
#             FOREIGN KEY (tweet_id) REFERENCES Tweets(tweet_id),
#             FOREIGN KEY (original_tweet_id) REFERENCES Tweets(tweet_id)
#             );
# """)
# # Note: The public metrics in this table refer to those of the retweets, not the original tweet.
# # Referenced tweets are all stored in the Tweets table (including text)

In [ ]:
# list all tables
con.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()

[('Retweets',), ('Tweets',), ('Sentiment',), ('Authors',)]

## Populate tables

### Populate the Authors table
author_id, name, username, description, location, verified, followers_count, following_count, tweet_count, listed_count, profile_image_url, url

In [ ]:
# con.execute("PRAGMA table_info('Authors')").fetchall()

[(0, 'author_id', 'TEXT', 1, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'username', 'TEXT', 0, None, 0),
 (3, 'description', 'TEXT', 0, None, 0),
 (4, 'location', 'TEXT', 0, None, 0),
 (5, 'verified', 'INTEGER', 0, None, 0),
 (6, 'followers_count', 'INTEGER', 0, None, 0),
 (7, 'following_count', 'INTEGER', 0, None, 0),
 (8, 'tweet_count', 'INTEGER', 0, None, 0),
 (9, 'listed_count', 'INTEGER', 0, None, 0),
 (10, 'profile_image_url', 'TEXT', 0, None, 0),
 (11, 'url', 'TEXT', 0, None, 0)]

In [ ]:
# # load authors from all_en_tweets.json
# con = sqlite3.connect(dir_path+"queen.db", timeout=500)
# cur = con.cursor()
# with open(dir_path+'all_en_tweets.json', 'r') as f:
#   i = 0
#   author_ids = set()
#   while True: 
#     if not i%200: # commit every 200 entries
#       con.commit()
#       con = sqlite3.connect(dir_path+"queen.db", timeout=500)
#       cur = con.cursor()
#     line = f.readline()
#     if not line: # if line is empty
#       break
#     t = json.loads(line) # tweet
#     a = t['author']
#     if a['id'] not in author_ids: # if author not added yet
#       cur.execute("""INSERT INTO Authors VALUES (?,?,?,?,?,?,?,?,?,?,?,?)""", 
#         ( a['id'], a['name'], a['username'], a['description'], a['location'] if 'location' in a else '', int(a['verified']), 
#         a['public_metrics']['followers_count'], a['public_metrics']['following_count'], a['public_metrics']['tweet_count'], 
#         a['public_metrics']['listed_count'], a['profile_image_url'], a['url'] if 'url' in a else '')
#       )
#       i += 1
#       author_ids.add(a['id'])
#   con.commit()

In [ ]:
# con.execute('select count(*) from Authors').fetchall()#, len(author_ids) # 2459678

[(2459678,)]

In [ ]:
# # load original authors from retweets.json (i.e. authors of tweets being retweeted)
# con = sqlite3.connect(dir_path+"queen.db", timeout=500)
# cur = con.cursor()
# author_ids = con.execute('select author_id from Authors').fetchall()
# author_ids = {a[0] for a in author_ids}
# with open(dir_path+'retweets.json', 'r') as f:
#   i = 0
#   while True: 
#     if not i%200: # commit every 200 entries
#       con.commit()
#       con = sqlite3.connect(dir_path+"queen.db", timeout=500)
#       cur = con.cursor()
#     line = f.readline()
#     if not line: # if line is empty
#       break
#     t = json.loads(line) # tweet
#     if 'referenced_tweets' in t: # if so, then t['referenced_tweets'] is always a list of len 1
#       if 'author' not in t['referenced_tweets'][0]:
#         continue # some really don't have author
#       a = t['referenced_tweets'][0]['author']
#     else:
#       continue
#     if a['id'] not in author_ids: # if author not added yet
#       cur.execute("""INSERT INTO Authors VALUES (?,?,?,?,?,?,?,?,?,?,?,?)""", 
#         ( a['id'], a['name'], a['username'], a['description'], a['location'] if 'location' in a else '', int(a['verified']), 
#         a['public_metrics']['followers_count'], a['public_metrics']['following_count'], a['public_metrics']['tweet_count'], 
#         a['public_metrics']['listed_count'], a['profile_image_url'], a['url'] if 'url' in a else '')
#       )
#       i += 1
#       author_ids.add(a['id'])
#   con.commit()

In [ ]:
# con.execute('select count(*) from Authors').fetchall() # 2460377

[(2460377,)]

In [ ]:
# con.execute('select count(distinct(location)) from Authors').fetchall() # 578740
con.execute("select count(*) from Authors where location=''").fetchall() # 829048 out of 2460377 don't have location

[(829048,)]

### Populate the Tweets table


In [ ]:
con.execute("PRAGMA table_info('Tweets')").fetchall()

[(0, 'tweet_id', 'TEXT', 1, None, 1),
 (1, 'author_id', 'TEXT', 1, None, 0),
 (2, 'tweet_text', 'TEXT', 1, None, 0),
 (3, 'country', 'TEXT', 0, None, 0),
 (4, 'retweet_count', 'INTEGER', 0, None, 0),
 (5, 'reply_count', 'INTEGER', 0, None, 0),
 (6, 'like_count', 'INTEGER', 0, None, 0),
 (7, 'quote_count', 'INTEGER', 0, None, 0)]

In [ ]:
# # From tweets.json
# con = sqlite3.connect(dir_path+"queen.db", timeout=10)
# cur = con.cursor()
# tweet_ids = set()
# with open(dir_path+'tweets.json', 'r') as f:
#   i = 0
#   while True: 
#     if not i%200: # commit every 200 entries
#       con.commit()
#       con = sqlite3.connect(dir_path+"queen.db", timeout=10)
#       cur = con.cursor()
#     line = f.readline()
#     if not line: # if line is empty
#       break
#     t = json.loads(line) # tweet
#     if t['id'] not in tweet_ids: # if tweet not added yet
#       cur.execute("""INSERT INTO Tweets VALUES (?,?,?,?,?,?,?,?)""", 
#         ( t['id'], t['author_id'], t['text'], t['geo']['country'] if 'geo' in t else '',
#           t['public_metrics']['retweet_count'], t['public_metrics']['reply_count'],
#           t['public_metrics']['like_count'], t['public_metrics']['quote_count'])
#       )
#       i += 1
#       tweet_ids.add(t['id'])
#   con.commit()

In [ ]:
con.execute('select count(*) from Tweets').fetchall() #, len(tweet_ids) # 795645

[(795645,)]

In [ ]:
con.execute("select count(*), count(distinct(country)) from Tweets where country!=''").fetchall()
# # out of 15583 tweets with country, there are 160 countries in total

[(15583, 160)]

In [ ]:
# # From retweets.json
# con = sqlite3.connect(dir_path+"queen.db", timeout=10)
# cur = con.cursor()
# with open(dir_path+'retweets.json', 'r') as f:
#   i = 0
#   while True:
#     if not i%200: # commit every 200 entries
#       con.commit()
#       con = sqlite3.connect(dir_path+"queen.db", timeout=10)
#       cur = con.cursor()
#     line = f.readline()
#     if not line: # if line is empty
#       break
#     t = json.loads(line) # tweet
#     if 'referenced_tweets' in t: # if the refereced tweet is present
#       if t['referenced_tweets'][0]['type']=='retweeted':
#         t = t['referenced_tweets'][0]
#       # else, there is either no referenced_tweets or it is 
#       # quoted (this has been checked), so we treat the parent 
#       # tweet object as a regular tweet
#     if t['id'] not in tweet_ids: # if tweet not added yet
#       # dealing with country. Better to use place id and their reverse geocoding 
#       geo = t['geo'] if 'geo' in t else ''
#       if geo!='':
#         geo = geo['country'] if 'country' in geo else ''
#       cur.execute("""INSERT INTO Tweets VALUES (?,?,?,?,?,?,?,?)""", 
#         ( t['id'], t['author_id'], t['text'], geo,
#           t['public_metrics']['retweet_count'], t['public_metrics']['reply_count'],
#           t['public_metrics']['like_count'], t['public_metrics']['quote_count'])
#       )
#     i += 1
#     tweet_ids.add(t['id'])         
#   con.commit()

In [ ]:
# con.execute('select count(*) from Tweets').fetchall() #, len(tweet_ids) # 795645
# con.execute("select count(*), count(distinct(country)) from Tweets where country!=''").fetchall() # nothing is added ...

[(15583, 160)]

### Populate the Retweet table

In [ ]:
con.execute("PRAGMA table_info('Retweets')").fetchall()

[(0, 'tweet_id', 'TEXT', 1, None, 1),
 (1, 'original_tweet_id', 'TEXT', 1, None, 0),
 (2, 'author_id', 'TEXT', 1, None, 0),
 (3, 'original_author_id', 'TEXT', 1, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'retweet_count', 'INTEGER', 0, None, 0),
 (6, 'reply_count', 'INTEGER', 0, None, 0),
 (7, 'like_count', 'INTEGER', 0, None, 0),
 (8, 'quote_count', 'INTEGER', 0, None, 0)]

In [ ]:
# # From retweets.json, took 1.5 hrs
# con = sqlite3.connect(dir_path+"queen.db", timeout=10)
# cur = con.cursor()
# retweet_ids = con.execute('select tweet_id from Retweets').fetchall()
# retweet_ids = {t[0] for t in retweet_ids}
# with open(dir_path+'retweets.json', 'r') as f:
#   i = 0
#   while True:
#     if not i%200: # commit every 200 entries
#       con.commit()
#       con = sqlite3.connect(dir_path+"queen.db", timeout=10)
#       cur = con.cursor()
#     line = f.readline()
#     if not line: # if line is empty
#       break
#     t = json.loads(line) # tweet
#     if t['id'] in retweet_ids:
#       continue # already added
#     if 'referenced_tweets' in t: # if the refereced tweet is present
#       if t['referenced_tweets'][0]['type']=='retweeted':
#         tt = t['referenced_tweets'][0]
#       else:
#         continue
#     else:
#       continue
#     if 'author_id' not in tt:
#       continue
#     cur.execute("""INSERT INTO Retweets VALUES (?,?,?,?,?,?,?,?,?)""", 
#       ( t['id'], tt['id'], t['author_id'], tt['author_id'], t['geo']['country'] if 'geo' in t else '',
#         t['public_metrics']['retweet_count'], t['public_metrics']['reply_count'],
#         t['public_metrics']['like_count'], t['public_metrics']['quote_count'])
#     )
#     i += 1         
#   con.commit()

In [ ]:
# con.execute("PRAGMA index_list(Retweets)").fetchall()
# con.execute('select count(tweet_id) from Retweets').fetchall() # Too slow
# con.execute("SELECT MAX(_ROWID_) FROM Retweets LIMIT 1").fetchall() # 3637269
con.execute("select count(*), count(distinct(country)) from Retweets where country!=''").fetchall() # nothing

[(0, 0)]

# Tweet Location

In [ ]:
import sqlite3
con = sqlite3.connect(dir_path+"queen.db", timeout=10)

In [ ]:
results = con.execute('''SELECT country, count(country) from Tweets where country!='' group by country''')

In [ ]:
results = list(results)

In [ ]:
df = pd.DataFrame(results).rename(columns={0:'country', 1:'count'})

In [ ]:
import plotly.express as px
fig = px.pie(values=df['count'], names=df['country'])
fig.update_layout(title=f"Tweet location by country (N={df['count'].sum()})", title_x=0.5)
fig.update_traces(textposition='inside',textinfo='percent+label', showlegend=False)

# Author Location: Country Identification
WARNING: DON'T USE THIS PART. THIS TAST WAS DONE SOMEWHERE ELSE. ONLY REFER TO THE OVERPASS API PART


- Country names https://pypi.org/project/pycountry/, 
https://github.com/flyingcircusio/pycountry
- US state names https://pypi.org/project/us/, https://github.com/unitedstates/python-us

In [7]:
# !pip install pycountry
# !pip install us

In [6]:
DC_STATEHOOD=1 # indicating that D.C. is a state in the us module
import pycountry
import us

In [4]:
# con = sqlite3.connect(dir_path+"queen.db", timeout=10)
# results = con.execute('''SELECT author_id, location from Authors where location!='' ''')
# results = list(results)

In [6]:
# df = pd.DataFrame(results).rename(columns={0:'author_id', 1:'location'})
# df_loc = df['location'].value_counts().reset_index().rename(columns={'index':'location', 'location':'count'})
# len(df_loc), df_loc['count'].sum()

(578739, 1631329)

In [ ]:
# df_loc.to_csv(dir_path+'author_locations_with_count_no_country.csv')

In [14]:
# Get country names and flags from pycountry
all_countries = list(pycountry.countries)
df_country = pd.DataFrame({'name': [c.name if 'name' in c._fields else None for c in all_countries],
              'common_name': [c.common_name if 'common_name' in c._fields else None for c in all_countries],
              'official_name': [c.official_name if 'official_name' in c._fields else None for c in all_countries],
              'alpha_2': [c.alpha_2 for c in all_countries],
              'flag': [c.flag for c in all_countries]})
countries_dict = {idx:{n for n in names if n is not None} for idx, names in df_country[['name','common_name','official_name']].iterrows()}

In [ ]:
# countries_dict
df_country.head(2)

,name,common_name,official_name,alpha_2,flag
0,Aruba,None,None,AW,🇦🇼
1,Afghanistan,None,Islamic Republic of Afghanistan,AF,🇦🇫


In [15]:
# Since we are matching substrings it is important to use longer strings first (e.g. Nigeria vs Niger)
countries_list = [(i,c) for i, names in countries_dict.items() for c in names]
countries_list.sort(key=lambda x: len(x[1]), reverse=True)
countries_list[:3]

[(79, 'United Kingdom of Great Britain and Northern Ireland'),
 (96, 'Hong Kong Special Administrative Region of China'),
 (135, 'Macao Special Administrative Region of China')]

## Round 1: cleaning non-letters
Declare the country of those <2 chars after cleaning as not found, but do this after we make use of the country flags


In [ ]:
# !pip install clean-text
from cleantext import clean

In [ ]:
def preprocess_location(s):
  s = clean(s, fix_unicode=False, to_ascii=False, no_emoji=True)
  s = re.sub(f'[{string.punctuation}0-9]', '', s)
  return s.strip()

In [ ]:
# df_loc['country'] = np.nan
# df_loc['location_original'] = df_loc['location']
# df_loc['location'] = df_loc.location_original.apply(preprocess_location)

## Round 2: Country exact matching

In [ ]:
# # Took 6 min
# # if any country name matches a substring of the location, identify that country
# for c_idx, c_name in countries_list: # for each country name
#   temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#   idx = temp[temp.location.str.contains(c_name, case=False)].index
#   df_loc.loc[idx, 'country']= df_country.loc[c_idx,'name']
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


(48802, 578739)

In [ ]:
# # if country flag is present, identify that country
# # # note that we must use location_original as the flags were removed in location
# for c_idx, c_flag in df_country.flag.items(): # for each country flag
#   temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#   idx = temp[temp.location_original.str.contains(c_flag, case=False)].index
#   df_loc.loc[idx, 'country'] = df_country.loc[c_idx, 'name']
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

(65881, 578739)

In [ ]:
# # Note: This should come after identifying US states
# # if <2 chars, then no country can be inferred
# temp = df_loc[df_loc.country.isna()]
# idx = temp[temp.location.apply(lambda s: len(s)<2)].index
# df_loc.loc[idx, 'country'] = 'NOT_FOUND'
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

(77144, 578739)

Manually looked into the keyword Georgia
- If the location is precisely 'Georgia' or contains 'Tbilisi' then we assume its the country of Georia
- Otherwise we assume it is the US state of Georgia

In [16]:
[(i,c) for i,c in enumerate(all_countries) if c.name=='United States' or c.name=='Georgia']

[(80,
  Country(alpha_2='GE', alpha_3='GEO', flag='🇬🇪', name='Georgia', numeric='268')),
 (234,
  Country(alpha_2='US', alpha_3='USA', flag='🇺🇸', name='United States', numeric='840', official_name='United States of America'))]

In [ ]:
# # First set it to the US
# df_loc.loc[df_loc[df_loc.location.str.contains('Georgia', case=False)].index, 'country'] = 'United States'
# df_loc.loc[df_loc[df_loc.location.str.contains(all_countries[234].flag)].index, 'country'] = 'United States'
# # Next deal with the country of Georgia
# df_loc.loc[df_loc[df_loc.location=='Georgia'].index, 'country'] = 'Georgia'
# df_loc.loc[df_loc[df_loc.location.str.contains('Tbilisi', case=False)].index, 'country'] = 'Georgia'
# df_loc.loc[df_loc[df_loc.location.str.contains(all_countries[80].flag)].index, 'country'] = 'Georgia'

### Second round: USA and UK
- Do this to take care of a large number of entries before going into matching with subdivision data, which is expensive
- US is not safe (e.g. AUS for Australia)
- US territories are treated separately

In [ ]:
# # UK
# temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
# uk_idx = temp[temp.location.str.contains('U\.?K\.?$', case=False)].index
# df_loc.loc[uk_idx, 'country']= 'United Kingdom'
# # USA
# temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
# usa_idx = temp[temp.location.str.contains('U\.?S\.?A\.?')].index
# df_loc.loc[usa_idx, 'country']= 'United States'
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

(83553, 578739)

### US States with abbreviations

In [17]:
us_states = us.states.STATES + [us.states.DC]
df_us_states = pd.DataFrame({'state_name': [s.name for s in us_states], 'state_abbr': [s.abbr for s in us_states]})
df_us_states.head(3)

,state_name,state_abbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ


In [18]:
us_states_list = list(df_us_states.state_name)
us_states_list.sort(key=lambda x: len(x), reverse=True)
us_states_list.remove('Georgia')

In [ ]:
# # D.C.
# temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
# idx = temp[temp.location.str.contains('Washington\s*,?\s*D\.?C\.?')].index
# df_loc.loc[idx, 'country']= 'United States'
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

(83553, 578739)

In [ ]:
# for s in us_states_list: # note that Georgia was removed
#   temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#   idx = temp[temp.location.str.contains(s)].index
#   df_loc.loc[idx, 'country']= 'United States'
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

(83553, 578739)

In [ ]:
# # Don't do this for consistency. Use Overpass instead
# # US territories 
# for terr in us.TERRITORIES:
#   temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#   idx = temp[temp.location.str.contains(terr.name)].index
#   df_loc.loc[idx, 'country']= terr.name
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

### US states abbreviation

In [ ]:
# for s in df_us_states.state_abbr:
#   temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#   idx = temp[temp.location.str.contains(f', {s}\.?(\W|$)')].index
#   df_loc.loc[idx, 'country']= 'United States'
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


(83553, 578739)

## pycountry subdivisions

In [8]:
subdiv = list(pycountry.subdivisions)
subdiv_dict = dict()
for s in subdiv:
  c_code = s.country_code
  if c_code not in subdiv_dict:
    subdiv_dict[c_code] = set()
  subdiv_dict[c_code].add(s.name)
len(subdiv_dict), sum([len(p) for p in subdiv_dict.values()])

(200, 5080)

In [ ]:
# for c_code in subdiv_dict:
#   assert c_code in set(df_country.alpha_2.values)

In [ ]:
# for c_code, place_set in subdiv_dict.items():
#   print(f'{c_code}, {len(place_set)} places to check ...')
#   for p in place_set:
#     if '[' in p: # this causes regex to complain; also make sense to remove them
#       p = re.sub('\[.*\]', '', p).strip()
#     temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#     idx = temp[temp.location.str.contains(f'(\W|^){p}(\W|$)', case=False)].index
#     c_idx = df_country[df_country['alpha_2']==c_code].index[0]
#     df_loc.loc[idx, 'country']= df_country.loc[c_idx, 'name']
# len(df_loc[~df_loc.country.isna()]), len(df_loc)

(144155, 578739)

In [ ]:
# df_loc.to_csv(dir_path+'location_with_pycountry.csv')

In [10]:
# df_loc = pd.read_csv(dir_path+'location_with_pycountry.csv')
# df_loc['location'] = df_loc.location.fillna('')

## Overpass API

- Tutorial: https://osm-queries.ldodds.com/tutorial/22-areas.osm.html
- OSM documentation: https://wiki.openstreetmap.org/wiki/Key:place

In [ ]:
# !pip install overpy

In [22]:
# import overpy
# api = overpy.Overpass() # 10k requests per day

In [ ]:
# # Example
# result = api.query("""area["ISO3166-1"="AE"]->.a1;
# (
#   node[place="state"](area.a1);
#   node[place="province"](area.a1);
#   node[place="region"](area.a1);
#   node[place="district"](area.a1);
#   node[place="county"](area.a1);
#   node[place="municipality"](area.a1);
#   node[place="city"](area.a1);
# );
# out tags;""")
# # node[place="town"](area.a1); # add this if more locations are needed
# [ [c.tags['name'],c.tags['name:en'] if 'name:en' in c.tags else None] for c in result.nodes]

In [ ]:
# # Get places in countries
# # with open(dir_path + 'osm_places.json', 'w') as f:
# with open(dir_path + 'osm_places.json', 'a') as f:
#   # for countrycode in df_country_names.alpha_2:
#     success = False
#     while not success:
#       try:
#         result = api.query(f"""area["ISO3166-1"="{countrycode}"]->.a1;
#           (
#             node[place="state"](area.a1);
#             node[place="province"](area.a1);
#             node[place="region"](area.a1);
#             node[place="district"](area.a1);
#             node[place="county"](area.a1);
#             node[place="municipality"](area.a1);
#             node[place="city"](area.a1);
#           );
#           out tags;""")
#         success = True
#       except overpy.exception.OverpassTooManyRequests:
#         print("OverpassTooManyRequests encountered when loading " + countrycode)
#         time.sleep(10)
#       except overpy.exception.OverpassGatewayTimeout
#         print("OverpassGatewayTimeout encountered when loading " + countrycode)
#         time.sleep(10)
#     p = {countrycode: [ [c.tags['name'] if 'name' in c.tags else None ,
#                  c.tags['name:en'] if 'name:en' in c.tags else None] 
#               for c in result.nodes] }
#     f.write(json.dumps(p)+'\n')
#     print(f'{countrycode} added')
#     time.sleep(10)

In [11]:
with open(dir_path + 'osm_places.json', 'r') as f:
  s = f.readlines()
places = [json.loads(p) for p in s]
places = {list(country.keys())[0]:list(country.values())[0] for country in places}
for k,v in places.items():
  if v==[]:
    places[k] = [set()]
places = {k:functools.reduce(lambda x,y: set(x).union(y),v) for k,v in places.items()}

In [33]:
# # Took forever
# for name, c_code in df_country[['name','alpha_2']].values:
#   if c_code not in places:
#     continue
#   place_set = places[c_code] # places in the country
#   print(f'Searching for matches in {name}, {len(place_set)} places ...', end='')
#   start_time = time.time()
#   num_matches = 0
#   for p in place_set:
#     temp = df_loc[df_loc.country.isna()] # work with rows not identified yet
#     idx = temp[temp.location.str.contains(f'(\W|^){p}(\W|$)', case=False)].index
#     df_loc.loc[idx, 'country']= name
#     num_matches += len(idx)
#   print(f'Took {int(time.time()-start_time)} seconds, found {num_matches} matches')

Searching for matches in Aruba, 0 places ...Took 0 seconds, found 0 matches
Searching for matches in Afghanistan, 830 places ...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # This is added back by InteractiveShellApp.init_path()


Took 510 seconds, found 346 matches
Searching for matches in Angola, 37 places ...Took 23 seconds, found 1 matches
Searching for matches in Anguilla, 0 places ...Took 0 seconds, found 0 matches
Searching for matches in Åland Islands, 4 places ...Took 2 seconds, found 2 matches
Searching for matches in Albania, 22 places ...Took 15 seconds, found 5 matches
Searching for matches in Andorra, 0 places ...Took 0 seconds, found 0 matches
Searching for matches in United Arab Emirates, 48 places ...Took 26 seconds, found 290 matches
Searching for matches in Argentina, 108 places ...Took 60 seconds, found 253 matches
Searching for matches in Armenia, 24 places ...Took 14 seconds, found 11 matches
Searching for matches in American Samoa, 1 places ...Took 0 seconds, found 0 matches
Searching for matches in Antarctica, 0 places ...Took 0 seconds, found 0 matches
Searching for matches in French Southern Territories, 0 places ...Took 0 seconds, found 0 matches
Searching for matches in Antigua and Ba

In [4]:
# df_loc.to_csv(dir_path + 'author_location_osm.csv')
df_loc= pd.read_csv(dir_path + 'author_location_osm.csv')

In [21]:
len(df_loc[df_loc.country.isna()]), len(df_loc), df_loc[~df_loc.country.isna()]['count'].sum()/df_loc['count'].sum()

(351941, 578739, 0.6442820546928302)

In [24]:
df_loc[df_loc.country.isna()][:100]

,Unnamed: 0,Unnamed: 0.1,location,count,country,location_original
3,3,3,sheher,11947,NaN,she/her
20,20,20,hehim,4059,NaN,he/him
30,30,30,earth,3613,NaN,Earth
38,38,38,shethey,2715,NaN,she/they
40,40,40,brooklyn ny,2558,NaN,"Brooklyn, NY"
41,41,41,theythem,2450,NaN,they/them
57,57,57,brasil,1859,NaN,Brasil
66,66,66,sheher,1656,NaN,she/her
72,72,72,hell,1603,NaN,Hell
92,92,92,sheher,1393,NaN,She/her


## More country


In [38]:
!pip install countryinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from countryinfo import CountryInfo

In [34]:
# country = CountryInfo('japan')

In [79]:
country_info_all = CountryInfo().all()

In [103]:
country_info_all = CountryInfo().all()
country_info = dict()
params_to_get = ['name', 'altSpellings', 'capital', 'nativeName', 'provinces']
for c in country_info_all:
  params = [p for p in params_to_get if p in country_info_all[c]]
  country_info[c] = {param: country_info_all[c][param] for param in params}
for c in country_info:
  if 'altSpellings' in country_info[c]:
    country_info[c]['altSpellings'] = country_info[c]['altSpellings'][1:] # the first entry is the two-letter code

In [103]:
country_info

In [ ]:
# temp = df_loc2[df_loc2.country.isna()]
# idx = temp[temp.location.str.contains('u\.?k\.?',case=False)].index
# df_loc2.loc[idx, 'country'] = 'United Kindgom'
# len(df_loc2[df_loc2.country.isna()])

50864

In [ ]:
# temp = df_loc2[df_loc2.country.isna()]
# idx = temp[temp.location.str.contains('n\.?y\.?c.?',case=False)].index
# df_loc2.loc[idx, 'country'] = 'United States'
# len(df_loc2[df_loc2.country.isna()])

50652

In [ ]:
# temp = df_loc2[df_loc2.country.isna()]
# idx = temp[temp.location.str.contains('日本|東京|大阪')].index
# df_loc2.loc[idx, 'country'] = 'Japan'
# len(df_loc2[df_loc2.country.isna()])

49329

In [ ]:
df_loc2[df_loc2.country.isna()]

,location,count,country,location_cleaned
72,Hell,1603,NaN,hell
130,hell,1097,NaN,hell
234,ประเทศไทย,601,NaN,ประเทศไทย
264,NY,531,NaN,ny
273,18,512,NaN,<number>
...,...,...,...,...
578699,𝟝𝕥𝕙 𝕎𝔸ℝ𝔻 𝕋.𝕏.𝕋ℝ𝕀𝕃𝕃,1,NaN,0𝕥𝕙 𝕎𝔸ℝ𝔻 𝕋𝕏𝕋ℝ𝕀𝕃𝕃
578704,ぽらんど,1,NaN,ぽらんど
578716,𝐬𝐡𝐞 / 𝐡𝐞𝐫,1,NaN,𝐬𝐡𝐞 𝐡𝐞𝐫
578730,・。.・゜✭・.・✫・,1,NaN,゜✭✫


In [ ]:
# len(df_loc), len(df_loc[~df_loc.country.isna()]), len(df_loc[~df_loc.country.isna()])/len(df_loc)
# # df_loc[~df_loc.country.isna()].country.value_counts()[:20]

## Count

In [ ]:
# df_loc[(df_loc.country.isna())&(df_loc['count']>300)]
# idx = df_loc[df_loc.location.str.contains('Korea')].loc[df_loc.country.isna()].index
# df_loc.loc[idx,'country'] = df_country_names.loc[122, 'name']